# One to One

In [1]:
import torch
import numpy as np
import gc

device = 'mps'

# RNN

In [3]:
class SimpleRNN(torch.nn.Module):
    def __init__(self, n_input=1, n_hidden=20, n_output=1):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.RNN(input_size=self.I,
                                hidden_size=self.H,
                                nonlinearity='tanh',
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0 = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])

        return out

# LSTM

In [4]:
class LSTM(torch.nn.Module):
    def __init__(self, n_input=1, n_hidden=20, n_output=1):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.rnn = torch.nn.LSTM(input_size=self.I,
                                 hidden_size=self.H,
                                 batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)
        c0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, (h_0, c_0) = self.rnn(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return out

# GRU

In [50]:
class GRU(torch.nn.Module):
    def __init__(self, n_input=1, n_hidden=20, n_output=1):
        super().__init__()
        self.I = n_input
        self.H = n_hidden
        self.O = n_output

        self.gru = torch.nn.GRU(input_size=self.I,
                                hidden_size=self.H,
                                batch_first=True)
        
        self.fc = torch.nn.Linear(self.H, self.O)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.H).to(device)

        out, h_0= self.gru(x, h0)
        out = self.fc(out[:, -1, :])

        return out

# 모델 생성

In [51]:
rnn_model = SimpleRNN(n_input=1, n_hidden=40, n_output=1).to(device)
lstm_model = LSTM(n_input=1, n_hidden=40, n_output=1).to(device)
gru_model = GRU(n_input=1, n_hidden=40, n_output=1).to(device)

# 학습

In [52]:
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 랜덤한 값을 생성 -> 1~4 사이의 값을 1000개 생성
y = np.square(X) # x의 제곱

train_X = torch.from_numpy(X.astype(np.float32)).to(device)
train_y = torch.from_numpy(y.astype(np.float32)).squeeze(-1).to(device)

crietrion = torch.nn.MSELoss()
rnn_optimizer = torch.optim.Adam(rnn_model.parameters())
lstm_optimizer = torch.optim.Adam(lstm_model.parameters())
gru_optimizer = torch.optim.Adam(gru_model.parameters())

In [8]:
for epoch in range(4000):
    outputs = rnn_model(train_X)
    loss = crietrion(outputs, train_y)

    rnn_optimizer.zero_grad()
    loss.backward()
    rnn_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 40.8957
Epoch [200/4000], Loss: 21.7805
Epoch [300/4000], Loss: 16.8765
Epoch [400/4000], Loss: 11.7607
Epoch [500/4000], Loss: 7.1105
Epoch [600/4000], Loss: 4.2321
Epoch [700/4000], Loss: 3.0310
Epoch [800/4000], Loss: 2.6126
Epoch [900/4000], Loss: 2.4124
Epoch [1000/4000], Loss: 2.2618
Epoch [1100/4000], Loss: 2.1256
Epoch [1200/4000], Loss: 1.9944
Epoch [1300/4000], Loss: 1.8635
Epoch [1400/4000], Loss: 1.7304
Epoch [1500/4000], Loss: 1.5935
Epoch [1600/4000], Loss: 1.4523
Epoch [1700/4000], Loss: 1.3074
Epoch [1800/4000], Loss: 1.1603
Epoch [1900/4000], Loss: 1.0131
Epoch [2000/4000], Loss: 0.8689
Epoch [2100/4000], Loss: 0.7309
Epoch [2200/4000], Loss: 0.6022
Epoch [2300/4000], Loss: 0.4855
Epoch [2400/4000], Loss: 0.3827
Epoch [2500/4000], Loss: 0.2948
Epoch [2600/4000], Loss: 0.2217
Epoch [2700/4000], Loss: 0.1628
Epoch [2800/4000], Loss: 0.1167
Epoch [2900/4000], Loss: 0.0815
Epoch [3000/4000], Loss: 0.0555
Epoch [3100/4000], Loss: 0.0369
Epoch [3200/4

In [34]:
X_test = torch.tensor([[[2.0]], [[3.0]], [[10.0]]], dtype=torch.float32).to(device)
outputs = rnn_model(X_test).cpu().tolist()

for i, re in enumerate(outputs):
    print(f'입력: {X_test[i][0].cpu()[0]}, 정답: {X_test[i][0].cpu()[0]**2}, 예측: {re[0]}')

# 학습에 없던 10의 경우 제대로 예측이 안 된다.

입력: 2.0, 정답: 4.0, 예측: 4.005005359649658
입력: 3.0, 정답: 9.0, 예측: 9.026866912841797
입력: 10.0, 정답: 100.0, 예측: 25.592641830444336


In [35]:
for epoch in range(4000):
    outputs = lstm_model(train_X)
    loss = crietrion(outputs, train_y)

    lstm_optimizer.zero_grad()
    loss.backward()
    lstm_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 68.4269
Epoch [200/4000], Loss: 33.3822
Epoch [300/4000], Loss: 18.4541
Epoch [400/4000], Loss: 13.2460
Epoch [500/4000], Loss: 8.2132
Epoch [600/4000], Loss: 4.4246
Epoch [700/4000], Loss: 2.5683
Epoch [800/4000], Loss: 1.8707
Epoch [900/4000], Loss: 1.5196
Epoch [1000/4000], Loss: 1.2565
Epoch [1100/4000], Loss: 1.0308
Epoch [1200/4000], Loss: 0.8303
Epoch [1300/4000], Loss: 0.6528
Epoch [1400/4000], Loss: 0.4992
Epoch [1500/4000], Loss: 0.3705
Epoch [1600/4000], Loss: 0.2671
Epoch [1700/4000], Loss: 0.1874
Epoch [1800/4000], Loss: 0.1289
Epoch [1900/4000], Loss: 0.0876
Epoch [2000/4000], Loss: 0.0597
Epoch [2100/4000], Loss: 0.0415
Epoch [2200/4000], Loss: 0.0297
Epoch [2300/4000], Loss: 0.0222
Epoch [2400/4000], Loss: 0.0173
Epoch [2500/4000], Loss: 0.0139
Epoch [2600/4000], Loss: 0.0114
Epoch [2700/4000], Loss: 0.0095
Epoch [2800/4000], Loss: 0.0079
Epoch [2900/4000], Loss: 0.0066
Epoch [3000/4000], Loss: 0.0055
Epoch [3100/4000], Loss: 0.0046
Epoch [3200/4

In [36]:
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device)
print(f"Input: 2.0, Output: {lstm_model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

Input: 2.0, Output: 3.9716007709503174, 정답: 4.0


In [53]:
for epoch in range(4000):
    outputs = gru_model(train_X)
    loss = crietrion(outputs, train_y)

    gru_optimizer.zero_grad()
    loss.backward()
    gru_optimizer.step()

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

Epoch [100/4000], Loss: 49.0133
Epoch [200/4000], Loss: 25.9639
Epoch [300/4000], Loss: 16.9823
Epoch [400/4000], Loss: 11.9726
Epoch [500/4000], Loss: 7.7389
Epoch [600/4000], Loss: 5.3142
Epoch [700/4000], Loss: 4.3695
Epoch [800/4000], Loss: 4.0927
Epoch [900/4000], Loss: 4.0582
Epoch [1000/4000], Loss: 3.8452
Epoch [1100/4000], Loss: 3.5047
Epoch [1200/4000], Loss: 3.4741
Epoch [1300/4000], Loss: 4.1647
Epoch [1400/4000], Loss: 3.2103
Epoch [1500/4000], Loss: 2.2244
Epoch [1600/4000], Loss: 1.5910
Epoch [1700/4000], Loss: 1.1581
Epoch [1800/4000], Loss: 0.8734
Epoch [1900/4000], Loss: 0.6783
Epoch [2000/4000], Loss: 0.5565
Epoch [2100/4000], Loss: 0.4856
Epoch [2200/4000], Loss: 0.4422
Epoch [2300/4000], Loss: 0.4122
Epoch [2400/4000], Loss: 0.3883
Epoch [2500/4000], Loss: 0.3673
Epoch [2600/4000], Loss: 0.3479
Epoch [2700/4000], Loss: 0.3300
Epoch [2800/4000], Loss: 0.3143
Epoch [2900/4000], Loss: 0.3012
Epoch [3000/4000], Loss: 0.2898
Epoch [3100/4000], Loss: 0.2811
Epoch [3200/4

In [54]:
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device)
print(f"Input: 2.0, Output: {gru_model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

Input: 2.0, Output: 5.418184280395508, 정답: 4.0
